In [6]:
import os

if "script" in os.getcwd():
    os.chdir("..")
if "notebooks" in os.getcwd():
    os.chdir("..")
print(os.getcwd())
import warnings
warnings.filterwarnings("ignore")

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from loguru import logger
from tqdm import tqdm


# torch.manual_seed(1234)
# np.random.seed(1234)

cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logger.info(f"Device : {device}")

def show_im(im, figsize=(20, 10)):
    plt.figure(figsize=figsize)
    if isinstance(im, list):
        plt.imshow(np.hstack([i for i in im]), cmap="gray")
    else:
        plt.imshow(im, cmap="gray")
    plt.show()


def show_multi_im(imgs: list, ratio=(5, 12), figsize=(20, 10)):
    index = 0
    f, axarr = plt.subplots(ratio[0], ratio[1], figsize=figsize)
    for r in range(ratio[0]):
        for c in range(ratio[1]):

            axarr[r, c].imshow(imgs[index], cmap="gray")
            index += 1
    plt.show()

def get_infor(data, lib="torch"):
    lib = torch if lib == "torch" else np
    return f"Max: {lib.max(data)} - Min: {lib.min(data)}"

/home/nguyentu/Documents/ThacSi/ImageDenoise


In [7]:
from script.demo_inference_configs import strategy_configs, sigmas
from script.demo_inference_functions import initialize_model, create_data, DemoUI

In [8]:
strategy_configs = {
    "4": {
        "model": ["dnresnet"],
        "model_configs": {
            "layers": 10,
            "input_channels": 2,
            "out_channels": 1,
            "feature": 192,
            "weigth_path": "result/layers_dnresnet10_Falseclip_Truenormalize_sigrange10_10_image_withMap/model_040.pth",
        },
        "range_sigma": sigmas,
        "step": 5,
        "data_path": "data",
        "n_workers": 8,
        "batch_size": 1,
        "normalize": True,
        "clip": False,
        "device": 'cuda:0',
        "output_mode": 'image',
        "with_map": True
    },
}


In [9]:
import glob
data_paths = glob.glob(f"data/test/origin/*.npy")

In [10]:
demo_ui = DemoUI(configs=strategy_configs)

In [11]:
saved_folder="report/LV_ThS_TuPhuongNguyen/paper2"
os.makedirs(saved_folder, exist_ok=True)

In [12]:
sigmas = [10, 35, 55]
noise_map_sigmas = [i*5 for i in range(15)]

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]

In [ ]:
import pandas as pd

sigma_values = []
psnr_values = []
ssim_values = []
image_paths = []
origin_folder = f"{saved_folder}/origin"
os.makedirs(origin_folder,exist_ok=True)

for sigma in sigmas:
    sigma_folder = f"{saved_folder}/sigma_{sigma}"  
    os.makedirs(sigma_folder,exist_ok=True)
    logger.success(f"---------------SIGMA={sigma}---------------")
    for data_path in tqdm(data_paths, desc=f"Sigma {sigma}"):
        for noise_map_sigma in noise_map_sigmas:
            number_data_path = os.path.basename(data_path)
            demo_ui.run(number_data_path=number_data_path, sigma=sigma, run_flag=True, create_image=False, create_noise_map=True, noise_map_sigma = noise_map_sigma)
            result = demo_ui.get_result()
            ori_image = result["image"]["Original"]
            noise_image = result["image"]["Noise"]
            denoise_image = result["image"]["Strategy4"]
    
            psnr = result["psnr"]["Strategy4"]
            ssim = result["ssim"]["Strategy4"]
            image_path = os.path.basename(data_path).replace(".npy", ".png")
            sigma_values.append(sigma)
            psnr_values.append(psnr)
            ssim_values.append(ssim)
            image_paths.append(image_path)
    
            cv2.imwrite(f"{origin_folder}/{image_path}", ori_image)
            cv2.imwrite(f"{sigma_folder}/{image_path}".replace(".png", "_noise.png"), noise_image)
            cv2.imwrite(f"{sigma_folder}/map_{noise_map_sigma}_{image_path}".replace(".png", "_denoise.png"), denoise_image)

# Create a DataFrame to store the data
df = pd.DataFrame({
    'Sigma': sigma_values,
    'PSNR': psnr_values,
    'SSIM': ssim_values,
    'Image_Path': image_paths
})

# Save the DataFrame to an Excel file
df.to_excel(f"{saved_folder}/metrics_results.xlsx", index=False)